<a href="https://colab.research.google.com/github/aleklearn/repo/blob/main/generative-ai/lc_rag_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sources:
- https://python.langchain.com/docs/integrations/llms/llamacpp
- https://github.com/abetlen/llama-cpp-python
- https://github.com/ggerganov/llama.cpp
- https://huggingface.co/


### Dependecies

In [1]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 24.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.28-cp310-cp310-manylinux_2_35_x86_64.whl size=8779231 sha256=249d26ebb10dbb6db86494b5cc0cb383dd3243101355730c37f94fcd9c51eb41
  Stored in directory: /root/.cache/pip/wheels/93/6e/a9/478cce089dc2a082bdcffe468a1c65465c91b25d911b30da82
Successfully built llama-cpp-python


In [2]:
!pip install -Uq langchain langchainhub pypdf sentence-transformers chromadb huggingface-hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 5.5 MB/s et

In [3]:
!nvidia-smi

Fri Jan 12 03:51:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Load PDF and Split

In [4]:
# Load THE BELMONT REPORT pdf
!wget https://www.hhs.gov/ohrp/sites/default/files/the-belmont-report-508c_FINAL.pdf

--2024-01-12 03:52:15--  https://www.hhs.gov/ohrp/sites/default/files/the-belmont-report-508c_FINAL.pdf
Resolving www.hhs.gov (www.hhs.gov)... 23.60.62.82, 2600:1409:12:3b2::219c, 2600:1409:12:3b8::219c
Connecting to www.hhs.gov (www.hhs.gov)|23.60.62.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123683 (121K) [application/pdf]
Saving to: ‘the-belmont-report-508c_FINAL.pdf’

the-belmont-report- 100%[===================>] 120.78K  --.-KB/s    in 0.06s   

2024-01-12 03:52:15 (2.05 MB/s) - ‘the-belmont-report-508c_FINAL.pdf’ saved [123683/123683]



In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

pdf_file = "/content/the-belmont-report-508c_FINAL.pdf"

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

pdf_loader = PyPDFLoader(pdf_file)

all_splits = pdf_loader.load_and_split(text_splitter)
len(all_splits)

89

### Embedding

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': 'cpu'})

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

### Vector Store

In [7]:
from langchain.vectorstores import Chroma

# Create vectorstore
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="./chroma_db")

### Retrieval

In [8]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [9]:
question = "Identify the three principles discussed in the Belmont Report."

retrieved_docs = retriever.get_relevant_documents(question)

len(retrieved_docs)

4

In [10]:
retrieved_docs

[Document(page_content="The Belmont Report attempts to summarize the basic ethical principles identified by the Commission in the course of \nits deliberations. It is the outgrowth of an intensive four-day period of discussions that were held in February 1976 at \nthe Smithsonian Institution's Belmont Conference Center supplemented by the monthly deliberations of the \nCommission that were held over a period of nearly four years. It is a statement of basic ethical principles and", metadata={'page': 0, 'source': '/content/the-belmont-report-508c_FINAL.pdf'}),
 Document(page_content="the Belmont Report be adopted in its entirety, as a statement of the Department's policy. The Department requests \npublic comment on this recommendation.", metadata={'page': 0, 'source': '/content/the-belmont-report-508c_FINAL.pdf'}),
 Document(page_content='fulfilling this part o f its charge, is available as DHEW Publication No. (OS) 78-0013 and No. (OS) 78 -0014, for sale by \nthe Superintendent of Docum

### Prompt

In [11]:
from langchain import hub

# Prompt
rag_prompt_llama = hub.pull("rlm/rag-prompt-llama")
rag_prompt_llama

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="[INST]<<SYS>> You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.<</SYS>> \nQuestion: {question} \nContext: {context} \nAnswer: [/INST]"))])

### Model

https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/blob/main/llama-2-13b-chat.Q4_K_M.gguf


In [12]:
!huggingface-cli download TheBloke/Llama-2-13B-chat-GGUF llama-2-13b-chat.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False


Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_K_M.gguf to /root/.cache/huggingface/hub/tmpex5aif_z
llama-2-13b-chat.Q4_K_M.gguf: 100% 7.87G/7.87G [01:22<00:00, 95.6MB/s]
./llama-2-13b-chat.Q4_K_M.gguf


In [13]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

model_path = "/content/llama-2-13b-chat.Q4_K_M.gguf"

n_gpu_layers = 40  # Change this value based on your model and your GPU VRAM pool.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=model_path,
    temperature=0.8,
    max_tokens=2000,
    top_p=1,
    n_ctx=4000,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [14]:
!nvidia-smi

Fri Jan 12 04:04:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0              28W /  70W |   7721MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Chain

In [15]:
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import StrOutputParser

# Stuff
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt_llama
    | llm
    | StrOutputParser()
)

In [16]:
question = "Identify the three principles discussed in the Belmont Report."
rag_chain.invoke(question)

  Sure! Based on the retrieved context, the three principles discussed in the Belmont Report are:

1. Respect for persons: This principle emphasizes that individuals should be treated with respect and dignity, regardless of their status or condition.
2. Beneficence: This principle requires researchers to act in the best interests of the subjects they are studying, and to minimize harm whenever possible.
3. Justice: This principle ensures that the benefits and burdens of research are distributed fairly and equitably among all parties involved.

'  Sure! Based on the retrieved context, the three principles discussed in the Belmont Report are:\n\n1. Respect for persons: This principle emphasizes that individuals should be treated with respect and dignity, regardless of their status or condition.\n2. Beneficence: This principle requires researchers to act in the best interests of the subjects they are studying, and to minimize harm whenever possible.\n3. Justice: This principle ensures that the benefits and burdens of research are distributed fairly and equitably among all parties involved.'